In [7]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

In [2]:
def create_regressor_columns_string(columns):
  # Join all independent variables to define the formula used by the model
  regressor_columns = list(filter(lambda x: x != 'impression_count',columns))
  regressor_columns_string = "+".join(regressor_columns)
  return regressor_columns_string

In [38]:
def regression(path):
    # Load data
    regression_df_pd = pd.read_csv(path)
    regression_df_pd = regression_df_pd.drop('tweet_text', axis=1)
    
    # Apply log transformation to dependent variable
    regression_df_pd['impression_count'] = regression_df_pd['impression_count'].apply(lambda x: np.log(1+x))

    # Define formula without cross terms
    regressor_columns_string = create_regressor_columns_string(regression_df_pd.columns)

    # Fit model
    mod = smf.ols(formula=f'impression_count ~ {regressor_columns_string}', data=regression_df_pd)
    res = mod.fit()

    # Get p-values
    p_values = res.pvalues
    nb_variables_before = len(p_values)
    
    # Filter p-values < 0.05
    p_values = p_values[p_values < 0.05]
    nb_variables_after = len(p_values)

    # Get significant variables
    significant_variables = list(p_values.index)
    # Remove intercept
    significant_variables.remove('Intercept')

    print(f'Number of discarded variables: {nb_variables_before - nb_variables_after}')
    print(f'Significant variables ({len(significant_variables)}): {significant_variables}')

    # Apply log transformation to significant variables if "dummy" is not in the name
    for var in significant_variables:
        if 'dummy' not in var:
            regression_df_pd[var] = regression_df_pd[var].apply(lambda x: np.log(1+x))

    # Perform new regression with significant variables
    regressor_columns_string_new = create_regressor_columns_string(significant_variables)
    mod_new = smf.ols(formula=f'impression_count ~ {regressor_columns_string_new}', data=regression_df_pd)
    res_new = mod_new.fit()

    return res, res_new

In [39]:
path_frech_celebrities = '../data/regression/french_celebrities_regression_df.csv'
res_french_celebrities = regression(path_frech_celebrities)

display(res_french_celebrities[0].summary())
display(res_french_celebrities[1].summary())

Number of discarded variables: 8
Significant variables (15): ['dummy_entertainmentleisurebusiness', 'dummy_francepolitics', 'dummy_jeanlucmlenchon', 'dummy_nikosaliagas', 'dummy_staderennaisfc', 'dummy_tweet_period_afternoon', 'dummy_tweet_period_morning', 'dummy_tweet_period_night', 'hashtags_count', 'mentions_count', 'tweet_external_urls_count', 'tweet_length', 'tweet_medias_count', 'tweet_sentiment', 'followers_count']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       impression_count   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     505.5
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        15:49:20   Log-Likelihood:                -42875.
No. Observations:               16151   AIC:                         8.579e+04
Df Residuals:                   16129   BIC:                         8.596e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              1.4278      0.050     28.773      0.000       1.331       1.525
dummy_celebrities                     -0.2276      0.446     -0.510      0.610      -1.102       0.647
dummy_entertainment                   -0.1144      0.131     -0.871      0.384      -0.372       0.143
dummy_entertainmentleisurebusiness    -0.3983      0.114     -3.508      0.000      -0.621      -0.176
dummy_francepolitics                   0.5358      0.184      2.919      0.004       0.176       0.896
dummy_jeanlucmlenchon                 -0.3570      0.159     -2.245      0.025      -0.669      -0.045
dummy_nikosaliagas                    -1.3781      0.118    -11.696      0.000      -1.609      -1.147
dummy_politicalfigures                -0.1088      0.150     -0.726      0.468      -0.402       0.185
dummy_soccer                          -0.3641      0.335     -1.086      0.278      -1.022       0.293
dummy_sports                          -0.1739      0.368     -0.473      0.636      -0.895       0.547
dummy_sportsfitnessbusiness            0.1431      0.369      0.388      0.698      -0.581       0.867
dummy_staderennaisfc                   6.3263      0.301     21.000      0.000       5.736       6.917
dummy_tvhosts                          0.2983      0.231      1.291      0.197      -0.155       0.751
dummy_tvstars                          0.2983      0.231      1.291      0.197      -0.155       0.751
dummy_tweet_period_afternoon           0.4064      0.043      9.550      0.000       0.323       0.490
dummy_tweet_period_morning             0.5269      0.045     11.686      0.000       0.438       0.615
dummy_tweet_period_night               0.4945      0.040     12.410      0.000       0.416       0.573
hashtags_count                        -0.0906      0.032     -2.798      0.005      -0.154      -0.027
mentions_count                        -0.3611      0.016    -22.806      0.000      -0.392      -0.330
tweet_external_urls_count              3.7477      0.071     52.493      0.000       3.608       3.888
tweet_length                           0.0084      0.000     19.919      0.000       0.008       0.009
tweet_medias_count                     1.7511      0.050     35.147      0.000       1.653       1.849
tweet_sentiment                        1.0858      0.083     13.141      0.000       0.924       1.248
followers_count                     4.922e-07   2.28e-08     21.542      0.000    4.47e-07    5.37e-07
==============================================================================
Omnibus:                      754.685   Durbin-Watson:                   1.418
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              409.995
Skew:                           0.228   Prob(JB):                

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       impression_count   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     911.5
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        15:49:20   Log-Likelihood:                -42253.
No. Observations:               16151   AIC:                         8.454e+04
Df Residuals:                   16136   BIC:                         8.465e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                          -3.974e+11   4.54e+11     -0.876      0.381   -1.29e+12    4.92e+11
dummy_entertainmentleisurebusiness    -0.2885      0.102     -2.835      0.005      -0.488      -0.089
dummy_francepolitics                   1.0016      0.111      9.018      0.000       0.784       1.219
dummy_jeanlucmlenchon                 -0.7716      0.158     -4.878      0.000      -1.082      -0.462
dummy_nikosaliagas                    -1.4667      0.119    -12.364      0.000      -1.699      -1.234
dummy_staderennaisfc                   4.0929      0.124     33.038      0.000       3.850       4.336
dummy_tweet_period_afternoon        3.974e+11   4.54e+11      0.876      0.381   -4.92e+11    1.29e+12
dummy_tweet_period_morning          3.974e+11   4.54e+11      0.876      0.381   -4.92e+11    1.29e+12
dummy_tweet_period_night            3.974e+11   4.54e+11      0.876      0.381   -4.92e+11    1.29e+12
hashtags_count                        -0.1853      0.065     -2.862      0.004      -0.312      -0.058
mentions_count                        -2.3159      0.055    -41.772      0.000      -2.425      -2.207
tweet_external_urls_count              5.1224      0.105     48.789      0.000       4.917       5.328
tweet_length                          -0.2700      0.044     -6.176      0.000      -0.356      -0.184
tweet_medias_count                     3.4815      0.085     40.736      0.000       3.314       3.649
tweet_sentiment                        0.7161      0.078      9.155      0.000       0.563       0.869
followers_count                        0.3225      0.022     14.407      0.000       0.279       0.366
==============================================================================
Omnibus:                      268.185   Durbin-Watson:                   1.424
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              261.841
Skew:                           0.284   Prob(JB):                     1.39e-57
Kurtosis:                       2.742   Cond. No.                     4.70e+14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.33e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [57]:
regression_stats_table = res.summary().tables[1].data[1:]
header = res.summary().tables[1].data[0]

regression_stats_table = pd.DataFrame(regression_stats_table, columns=header)
regression_stats_table['coef'] = regression_stats_table['coef'].astype(float)
regression_stats_table['P>|t|'] = regression_stats_table['P>|t|'].astype(float)
regression_stats_table_significant = regression_stats_table[regression_stats_table['P>|t|'] < 0.05]
significant_regression_columns = regression_stats_table[''].values.tolist()[1:]

['dummy_celebrities',
 'dummy_entertainment',
 'dummy_entertainmentleisurebusiness',
 'dummy_francepolitics',
 'dummy_jeanlucmlenchon',
 'dummy_nikosaliagas',
 'dummy_politicalfigures',
 'dummy_soccer',
 'dummy_sports',
 'dummy_sportsfitnessbusiness',
 'dummy_staderennaisfc',
 'dummy_tvhosts',
 'dummy_tvstars',
 'dummy_tweet_period_afternoon',
 'dummy_tweet_period_morning',
 'dummy_tweet_period_night',
 'hashtags_count',
 'mentions_count',
 'tweet_external_urls_count',
 'tweet_length',
 'tweet_medias_count',
 'tweet_sentiment',
 'followers_count']

In [ ]:
regressor_columns = list(filter(lambda x: x not in [
                         'tweet_external_urls_count', 'dummy_tweet_period_night', 'dummy_joemanchin', 'dummy_chrismurphy', 'dummy_financialservicesbusiness', 'dummy_inflationintheunitedstates', 'dummy_joebiden', 'dummy_politicalfigures', 'dummy_northcarolina', 'dummy_tedcruz', 'dummy_sportsfitnessbusiness', 'dummy_unitedstatescongress', 'dummy_markwarner', 'dummy_politicalnews', 'dummy_chriscoons'], regression_df_pd.columns))

regressor_columns_string = create_regressor_columns_string(
    regressor_columns)
mod = smf.ols(
    formula=f'impression_count ~ {regressor_columns_string}', data=regression_df_pd)
res = mod.fit()
print(res.summary())